## Import modules

In [1]:
import pytesseract as pst
import PIL.Image as pi
import cv2
from pytesseract import Output
from wand.image import Image as wii
from pdf2image import convert_from_path

import numpy as np
import pandas as pd

import os
import string
import IPython.display as Ipd
from glob import glob as gg

# Data preporition

### Converting PDF to PNG

In [2]:
poppler_path = r"C:\Users\Elara\Downloads\Release-23.01.0-0\poppler-23.01.0\Library\bin"
saving_folder = "./png/"

In [8]:
i = 1

while i < 10:
    pdf_path = f"./pdf/{i}.pdf"
    pages = convert_from_path(pdf_path = pdf_path, poppler_path = poppler_path)

    j = 1
    for page in pages:
        img_name = f"Doc-{i}_Img-{j}.png"
        page.save(os.path.join(saving_folder, img_name), "PNG")
        j += 1
    i += 1

### Changing of color style of image

In [35]:
# ny = wii(filename = './png/Lab 2.jpg')
# ny.transform_colorspace('gray')
# ny.sketch(0, 1, 0)
# ny.save(filename = 'test1.jpg')

### Creating borders around word

In [36]:
# Ipd.Image('./png/Lab 2.jpg')

### Configuration of text recognition

In [37]:
my_config = r"--psm 3 --oem 3 -l eng" # --psm 3, 6

In [38]:
# img = cv2.imread('./png/img-1.png')
# height, width, _ = img.shape

In [39]:
# boxes = pst.image_to_boxes(img, config=my_config)
# for box in boxes.splitlines():
#     box = box.split(" ")
#     img = cv2.rectangle(img, (int(box[1]), height - int(box[2])), (int(box[3]), height - int(box[4])), (0, 255, 0))

In [40]:
# data = pst.image_to_data(img, config=my_config, output_type=Output.DICT)

### Returning text from image

In [41]:
text = pst.image_to_string(pi.open('./png/img-1.png'), config=my_config)
text_list = text.split()

### Droping non important info

In [42]:
del text_list[:text_list.index('ABSTRACT') + 1]

## Fuctions

### Drop non word

In [43]:
stop_words = ['the', 'of', 'is', 'to', 'and', 'in', 'for', 'as', 'are', 'that', 'on', 
              'an', 'it', 'be', 'this', 'with', 'from', 'by', 'which', 'their', 'can',
              'at', 'where', 'use', 'when', 'not', 'if', 'all', 'such', 'there', 'or',
              'you', 'they', 'oo', 'each', 'have', 'them', 'has', 'these', 'out', 'then',
              'etc', 'time', 'also', 'was', 'we', 'thus', 'about', 'should', 'lets', 'will']

In [44]:
def drop_non_word(text_list):
    for word in list(text_list):
        if (ord(word[0]) < 65 or (ord(word[0]) > 90 and ord(word[0]) < 97) or ord(word[0]) > 122) or '_' in word or len(word) < 2:
            text_list.remove(str(word))

### Save words in Uppercase

In [45]:
def uppercase_words(text_list):
    global uppercases
    uppercases = []
    
    for word in list(text_list):
        if word.isupper() and word != "INTRODUCTION" and word != "CONCLUSION" and word != "REFERENCES:":
            text_list.remove(word)
            uppercases.append(word)

### Drop Punctuation

In [46]:
def drop_punctuation(text_list):
    global clean_list
    clean_list = []
    
    for word in list(text_list):
        if str(word) not in stop_words:
            a = str(word).lower()
            clean_list.append(a.translate(str.maketrans('', '', string.punctuation)))
    
    str1 = ' '.join(clean_list)
    clean_list = str1.split()

### Drop duplicates

In [47]:
def drop_duplicates(any_list):
    global unique
    unique = []
    unique = [*set(any_list)]

    for word in list(any_list):
        if (ord(word[0]) < 65 or (ord(word[0]) > 90 and ord(word[0]) < 97) or ord(word[0]) > 122) or '_' in word:
            any_list.remove(str(word))
        if len(word) < 2:
            any_list.remove(str(word))
    
    return len(unique), unique.sort()

In [48]:
def word_counts(any_list):
    global my_df
    my_df = pd.value_counts(np.array(any_list))

### Main combine function

In [49]:
def cleaning_text(text_list):
    drop_non_word(text_list)
    uppercase_words(text_list)
    drop_punctuation(text_list)

In [50]:
cleaning_text(text_list)
text_list = clean_list

In [51]:
abstract = [text_list[:text_list.index('keywords')]]

In [52]:
text_list = text_list[text_list.index('keywords') + 1:]
keywords = text_list[:text_list.index('introduction')]

In [53]:
content = []
content = text_list[text_list.index('introduction') + 1:]

In [54]:
clean_list.clear()

In [55]:
# cv2.imshow("img", img)
# cv2.waitKey(0)

In [56]:
i = 2

while i < 18:
    path = './png/img-'+ str(i) + '.png'
    text = pst.image_to_string(pi.open(path), config=my_config)
    text_list = text.split()
    cleaning_text(text_list)
    text_list = clean_list
    del text_list[:text_list.index('jatitorg') + 1]
    content.extend(text_list)
    clean_list.clear()
    i += 1

In [57]:
path = './png/img-18.png'
text = pst.image_to_string(pi.open(path), config=my_config)
text_list = text.split()
cleaning_text(text_list)
text_list = clean_list
start = text_list.index('jatitorg') + 1
conclusion = text_list.index('conclusion') + 1
stop = text_list.index('references')
conclusion_text = text_list[conclusion:stop]
del text_list[:start]
text_list = text_list[start:conclusion]
content.extend(text_list)

In [58]:
all_the_words = []
all_the_words.extend(uppercases)
all_the_words.extend(abstract)
all_the_words.extend(keywords)
all_the_words.extend(content)
all_the_words.extend(conclusion_text)

In [59]:
clean_list.clear()

In [60]:
sorted_words = []

for word in list(all_the_words):
    a = str(word).lower()
    sorted_words.append(a.translate(str.maketrans('', '', string.punctuation)))

str1 = ' '.join([str(i) for i in all_the_words])
sorted_words = str1.split()
    
sorted_words.sort()

In [61]:
drop_duplicates(sorted_words)

(1693, None)

In [62]:
word_counts(sorted_words)

df = pd.DataFrame(my_df)

# df.to_csv('./csv/CountOfWords', encoding='utf-8')


In [63]:
df

,0
information,160
concepts,145
ontology,120
the,116
systems,107
...,...
importance,1
impossible,1
improved,1
inaccuracies,1


In [64]:
print(unique)

["'accordingly',", "'achieve',", "'adjust',", "'aggregated',", "'algorithm',", "'and',", "'application',", "'approach',", "'article',", "'automated',", "'based',", "'build',", "'building',", "'business',", "'changes',", "'complication',", "'computational',", "'concepts',", "'consequently',", "'constant',", "'continues',", "'converters',", "'data',", "'decisions',", "'decrease',", "'depends',", "'develop',", "'developers',", "'difficult',", "'effective',", "'elements',", "'especially',", "'evaluating',", "'following',", "'forced',", "'goal',", "'heterogeneity',", "'heterogeneous',", "'imagine',", "'included',", "'includes',", "'information',", "'integrated',", "'integrating',", "'integration',", "'interaction',", "'interconnectedness',", "'its',", "'leads',", "'making',", "'management',", "'mathematical',", "'method',", "'model',", "'models',", "'need',", "'obtaining',", "'often',", "'ontological',", "'ontologies',", "'ontologies']", "'own',", "'problem',", "'processes',", "'programs',"

In [65]:
print(sorted_words)

['A.,', 'A.F.', 'E-ISSN:', 'EKAW', 'FQAS', 'H.A.', 'IAIS', 'ISSN:', 'K.I.', 'LNAI.', 'LNCS.', 'LNCS.', 'M.', 'M.,', 'M.A.', 'P.', 'P.', 'P.', 'P.', 'P.', 'RS', 'S.', 'V.', 'V.', 'Y.', 'Z.,', 'a2', 'a2', 'a2', 'a2', 'a2', 'ability', 'ability', 'ability', 'able', 'above', 'absence', 'absent', 'absolute', 'absolute', 'abstract', 'abstract', 'abstraction', 'accelerated', 'accelerated', 'accepted', 'accepted', 'accepts', 'access', 'access', 'access', 'access', 'access', 'access', 'access', 'access', 'access', 'accessing', 'accordance', 'accordance', 'according', 'according', 'according', 'according', 'accordingly', 'accordingly', 'account', 'account', 'account', 'account', 'account', 'account', 'account', 'account', 'account', 'account', 'accounting', 'accumulated', 'accumulating', 'achieve', 'achieve', 'achieved', 'achievement', 'action', 'action', 'action', 'actions', 'actively', 'activities', 'activities', 'activities', 'activities', 'activities', 'acute', 'ad', 'adapt', 'adapted', 'adap

In [66]:
print(len(sorted_words), len(all_the_words))

6181 6214


In [67]:
all_the_words

['ISSN:',
 'RS',
 'E-ISSN:',
 'IAIS',
 'M.,',
 'Y.',
 'LNCS.',
 'V.',
 'A.F.',
 'K.I.',
 'Z.,',
 'M.',
 'P.',
 'H.A.',
 'P.',
 'P.',
 'FQAS',
 'LNCS.',
 'V.',
 'P.',
 'A.,',
 'S.',
 'EKAW',
 'LNAI.',
 'P.',
 'M.A.',
 ['today',
  'difficult',
  'imagine',
  'without',
  'integrated',
  'automated',
  'information',
  'systems',
  'support',
  'various',
  'business',
  'processes',
  'obtaining',
  'aggregated',
  'information',
  'purpose',
  'making',
  'management',
  'decisions',
  'depends',
  'effective',
  'interaction',
  'information',
  'systems',
  'included',
  'structure',
  'business',
  'processes',
  'often',
  'automated',
  'using',
  'software',
  'solutions',
  'its',
  'own',
  'thirdparty',
  'developers',
  'regardless',
  'interconnectedness',
  'especially',
  'true',
  'universities',
  'with',
  'constant',
  'changes',
  'business',
  'processes',
  'developers',
  'forced',
  'adjust',
  'programs',
  'data',
  'models',
  'leads',
  'structural',
  'semanti